In [3]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [4]:
import nltk
from nltk.tokenize import word_tokenize

# Check if the resource exists
nltk.data.find('tokenizers/punkt')
print("Punkt tokenizer is installed correctly.")


Punkt tokenizer is installed correctly.


In [1]:
import sys
print(f"Python executable: {sys.executable}")
print(f"Python version: {sys.version}")


Python executable: c:\Users\user\Desktop\10 Acedamy W5\TELEGRAM_SCRAPPER\venv\Scripts\python.exe
Python version: 3.11.6 (tags/v3.11.6:8b6ee5b, Oct  2 2023, 14:57:12) [MSC v.1935 64 bit (AMD64)]


In [6]:
import asyncio
import logging
from telethon import TelegramClient
import csv
import os

# Set up logging
logging.basicConfig(level=logging.INFO, filename="scraper.log", filemode="w",
                    format="%(asctime)s - %(levelname)s - %(message)s")

# Configurations
DATA_FOLDER = r"C:\Users\user\Desktop\10 Acedamy W5\All Data"
CHANNELS_FILE = os.path.join(DATA_FOLDER, "channels_to_crawl.csv")
SCRAPED_FILE = os.path.join(DATA_FOLDER, "scraped_data.csv")
API_ID = 20173022  # Replace with your API ID
API_HASH = 'bab4a3351ed7634a8c1a3f8767fcf75c'  # Replace with your API Hash


def load_channels(file_path):
    """Load channel usernames from a CSV file."""
    try:
        with open(file_path, 'r') as f:
            # Ensure each line is a list of strings, then we can access the correct column (1 index) and strip
            return [line[1].strip() for line in csv.reader(f) if len(line) > 1]
    except FileNotFoundError:
        logging.error(f"Channels file not found at {file_path}")
        return []


async def scrape_channel(client, channel, writer):
    """Scrape messages from a Telegram channel."""
    try:
        async for message in client.iter_messages(channel, limit=100):
            writer.writerow([channel, message.sender_id, message.date, message.text])
    except Exception as e:
        logging.error(f"Error scraping channel {channel}: {e}")


async def scrape_telegram_data():
    """Main function to scrape Telegram data."""
    logging.info("Starting Telegram scraper...")
    channels = load_channels(CHANNELS_FILE)
    if not channels:
        logging.error("No channels to scrape.")
        return

    async with TelegramClient('scraper', API_ID, API_HASH) as client:
        with open(SCRAPED_FILE, 'w', newline='', encoding='utf-8') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(['channel', 'sender', 'timestamp', 'message'])
            for channel in channels:
                logging.info(f"Scraping channel: {channel}")
                await scrape_channel(client, channel, writer)

    logging.info("Scraping completed.")


# Use await for environments with an active event loop
if __name__ == "__main__":
    await scrape_telegram_data()


In [ ]:
task 2

In [9]:
import pandas as pd

# Define the entity types for labeling
ENTITY_TYPES = {
    "B-Product": "Product entity",
    "I-Product": "Product entity continuation",
    "B-LOC": "Location entity",
    "I-LOC": "Location entity continuation",
    "B-PRICE": "Price entity",
    "I-PRICE": "Price entity continuation",
    "O": "Other (no entity)"
}

# Example dataset (replace this with your dataset)
data = [
    ("Baby bottle እንግዲኛ አሁን ዋጋ 1000 ብር", "B-Product I-Product O B-PRICE I-PRICE O"),
    ("Addis Abeba በፍቅር እንቆቅልሽ", "B-LOC I-LOC O B-Product I-Product")
]

# Convert the data into a pandas DataFrame
df = pd.DataFrame(data, columns=["Message", "Label"])

# Function to save labeled data in CoNLL format
def save_conll_format(df, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            message = row['Message']
            labels = row['Label'].split()
            message_tokens = message.split()

            # Iterate over tokens and corresponding labels
            for token, label in zip(message_tokens, labels):
                f.write(f"{token} {label}\n")
            
            f.write("\n")  # Blank line between sentences/messages

    print(f"Labeled data saved in CoNLL format to {file_path}")

# Save the labeled data to CoNLL format
save_conll_format(df, "labeled_data.conll")


Labeled data saved in CoNLL format to labeled_data.conll


In [ ]:
task 3

In [4]:
import torch
print(torch.cuda.is_available())  # Should return False
print(torch.__version__)          # Confirm PyTorch version


False
2.5.1+cpu


In [5]:
import pandas as pd
from datasets import load_dataset

# Load dataset in CoNLL format
labeled_data_path = "C:\\Users\\user\\Desktop\\10 Acedamy W5\\All Data\\labeled_telegram_product_price_location.txt"
dataset = load_dataset("text", data_files={"train": labeled_data_path})

# Convert to DataFrame for inspection (optional)
df = pd.DataFrame(dataset["train"])
print(df.head())


c:\Users\user\Desktop\10 Acedamy W5\TELEGRAM_SCRAPPER\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 11 examples [00:00, 355.62 examples/s]

              text
0  Example_Group O
1             is O
2          being O
3      protected O
4             by O


In [8]:
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")


In [11]:
from transformers import AutoTokenizer
from datasets import Dataset

# Define a sample dataset
data = {"text": ["Hello, world!", "Transformers are powerful!"]}
dataset = Dataset.from_dict(data)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Define the tokenize_and_align_labels function
def tokenize_and_align_labels(examples):
    # 'examples' here is passed from the dataset.map() function
    tokenized_inputs = tokenizer(
        examples["text"],  # Accessing the 'text' key from the input examples
        padding="max_length",  # Pads all sequences to the same length
        truncation=True,       # Truncates sequences longer than max_length
        max_length=11,         # Set to the desired fixed length
    )
    return tokenized_inputs

# Apply the function to your dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Print results to verify tokenized output
print(tokenized_dataset["input_ids"])  # This will show the tokenized IDs
print(tokenized_dataset["attention_mask"])  # This will show the attention mask


Map: 100%|██████████| 2/2 [00:00<00:00, 111.79 examples/s]

[[0, 35378, 4, 8999, 38, 2, 1, 1, 1, 1, 1], [0, 11062, 82772, 7, 621, 113138, 38, 2, 1, 1, 1]]
[[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]


In [13]:
# Adding dummy labels for token classification (this would be task-specific)
labels = [
    [1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0],  # Example labels for the first sentence
    [1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0],  # Example labels for the second sentence
]

# Adding labels to the dataset
dataset = dataset.add_column("labels", labels)


In [14]:
from transformers import AutoModelForTokenClassification, Trainer, TrainingArguments

# Load the pre-trained XLM-RoBERTa model for token classification
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=3)  # Change num_labels for your task


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from datasets import Dataset
from transformers import Trainer, TrainingArguments

# Sample dataset
data = {"text": ["Hello, world!", "Transformers are powerful!"]}
dataset = Dataset.from_dict(data)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

# Add dummy labels to match the length of tokenized inputs (for testing purposes)
labels = [
    [1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0],  # Labels for the first sentence
    [1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0],  # Labels for the second sentence
]

# Tokenize the dataset
def tokenize_and_align_labels(examples):
    # Tokenizing the text while padding to max_length and truncating if necessary
    tokenized_inputs = tokenizer(
        examples["text"],
        padding="max_length",  # Ensures all sequences are padded to max_length
        truncation=True,       # Truncates if longer than max_length
        max_length=11,         # Set to your desired fixed length (11 in this case)
    )
    tokenized_inputs["labels"] = labels  # Add labels to the tokenized inputs
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Load pre-trained model for token classification
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base", num_labels=3)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # You can split this into train and eval datasets
    tokenizer=tokenizer
)

# Train the model
trainer.train()


Map: 100%|██████████| 2/2 [00:00<00:00, 170.31 examples/s]
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\user\AppData\Local\Temp\ipykernel_21620\2359476935.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.912069
2,No log,0.883808
3,No log,0.869913


TrainOutput(global_step=3, training_loss=0.9460399945576986, metrics={'train_runtime': 24.4349, 'train_samples_per_second': 0.246, 'train_steps_per_second': 0.123, 'total_flos': 33683089572.0, 'train_loss': 0.9460399945576986, 'epoch': 3.0})

In [20]:
# Assuming you have the 'test_dataset' already loaded (similar to how you loaded the training dataset)
# If not, load it from a file or create it as a Dataset object

# Example: If you have a list of examples as input
test_data = {"text": ["Test sentence 1", "Test sentence 2"]}
test_dataset = Dataset.from_dict(test_data)

# Apply the same tokenization process to the test dataset
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

# Ensure that the evaluation dataset is tokenized and ready
print(tokenized_test_dataset)


Map: 100%|██████████| 2/2 [00:00<00:00, 200.53 examples/s]

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 2
})


In [21]:
# Evaluate the model using the trainer
eval_results = trainer.evaluate(tokenized_test_dataset)

# Print the evaluation results
print(eval_results)


{'eval_loss': 0.9102998375892639, 'eval_runtime': 0.7184, 'eval_samples_per_second': 2.784, 'eval_steps_per_second': 1.392, 'epoch': 3.0}


In [22]:
model.save_pretrained("your_model_save_path")
tokenizer.save_pretrained("your_tokenizer_save_path")


('your_tokenizer_save_path\\tokenizer_config.json',
 'your_tokenizer_save_path\\special_tokens_map.json',
 'your_tokenizer_save_path\\sentencepiece.bpe.model',
 'your_tokenizer_save_path\\added_tokens.json',
 'your_tokenizer_save_path\\tokenizer.json')

In [23]:
tokenizer.save_pretrained("your_tokenizer_save_path")


('your_tokenizer_save_path\\tokenizer_config.json',
 'your_tokenizer_save_path\\special_tokens_map.json',
 'your_tokenizer_save_path\\sentencepiece.bpe.model',
 'your_tokenizer_save_path\\added_tokens.json',
 'your_tokenizer_save_path\\tokenizer.json')

In [ ]:
task 4

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# 1. Create a sample text classification dataset
data = {
    "text": ["I love machine learning!", "This is a bad product.", "Great customer service.", "Terrible experience.", "Amazing quality!"],
    "label": [1, 0, 1, 0, 1]  # 1 for positive, 0 for negative
}
dataset = Dataset.from_dict(data)

# 2. Tokenizer Setup
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=32)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 3. Model Setup
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# 4. Define TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",        # Output directory
    num_train_epochs=3,            # Number of epochs
    per_device_train_batch_size=8, # Batch size per device during training
    per_device_eval_batch_size=16, # Batch size for evaluation
    warmup_steps=500,              # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,             # Strength of weight decay
    logging_dir="./logs",          # Directory for storing logs
    logging_steps=10,
    evaluation_strategy="epoch",   # Evaluation strategy to use
    save_strategy="epoch",         # Save model every epoch
    load_best_model_at_end=True    # Load the best model at the end of training
)

# 5. Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,  # You can replace this with a separate test dataset
    tokenizer=tokenizer,
)

# 6. Train the Model
trainer.train()

# 7. Evaluate the Model
eval_results = trainer.evaluate()
print(eval_results)

# 8. Save the Model and Tokenizer
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")

# 9. Test the Model (for inference)
# Test data for prediction
test_data = ["I am very happy with this product!", "Worst purchase I've made."]
encoded_test_data = tokenizer(test_data, padding=True, truncation=True, max_length=32, return_tensors="pt")

# Make predictions
with torch.no_grad():
    outputs = model(**encoded_test_data)
    predictions = torch.argmax(outputs.logits, dim=-1)
    print("Predictions: ", predictions)


Map: 100%|██████████| 5/5 [00:00<00:00, 385.63 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\user\Desktop\10 Acedamy W5\TELEGRAM_SCRAPPER\venv\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\user\AppData\Local\Temp\ipykernel_21620\2940384145.py:40: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,0.743370
2,No log,0.742265
3,No log,0.740112


{'eval_loss': 0.7401124835014343, 'eval_runtime': 0.4306, 'eval_samples_per_second': 11.611, 'eval_steps_per_second': 2.322, 'epoch': 3.0}
Predictions:  tensor([1, 1])


In [ ]:
task 5

In [28]:
import shap
import numba
import numpy as np
print(shap.__version__)
print(numba.__version__)
print(np.__version__)


ImportError: Numba needs NumPy 2.1 or less. Got NumPy 2.2.

In [27]:
import shap
import torch
from transformers import AutoModelForTokenClassification, AutoTokenizer
import numpy as np

# Load the model and tokenizer
model_name = "your_finetuned_model"  # Replace this with your actual model name or path
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example text to explain
text = "Hawking's work on black holes revolutionized physics."

# Tokenize the input text
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Get predictions from the model
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits
predictions = torch.argmax(logits, dim=-1)

# Map predictions to labels
labels = model.config.id2label  # Dictionary mapping the model's label IDs to label names
predicted_labels = [labels[prediction.item()] for prediction in predictions[0]]

# Prepare the explanation function for SHAP
def transformer_predict(input_ids):
    # Generate the input batch and get predictions
    input_batch = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
    encodings = tokenizer(input_batch, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**encodings)
    logits = outputs.logits
    return logits

# Use SHAP to explain the model's decision
explainer = shap.Explainer(transformer_predict, tokenizer)
shap_values = explainer(inputs["input_ids"])

# Visualize the SHAP values
shap.initjs()
shap.summary_plot(shap_values, inputs["input_ids"])

# Print predictions to verify the output
print("Predictions:", predicted_labels)


ImportError: Numba needs NumPy 2.1 or less. Got NumPy 2.2.

In [ ]:
Task 3

In [10]:
def preprocess_data(dataset):
    from transformers import XLMRobertaTokenizerFast
    
    tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
    max_length = 128  # Define the maximum sequence length

    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["tokens"],
            truncation=True,
            padding="max_length",
            max_length=max_length,
            is_split_into_words=True,
        )
        
        labels = []
        for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=batch_idx) for batch_idx in range(len(tokenized_inputs["input_ids"]))):
            example_labels = examples["ner_tags"][i]
            aligned_labels = []
            previous_word_id = None
            for word_id in word_ids:
                if word_id is None:
                    aligned_labels.append(-100)  # Ignore special tokens
                elif word_id != previous_word_id:
                    aligned_labels.append(example_labels[word_id])  # Original word label
                else:
                    aligned_labels.append(-100)  # Ignore subword tokens
                previous_word_id = word_id
            
            labels.append(aligned_labels)

        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    # Apply tokenization and alignment
    dataset = dataset.map(tokenize_and_align_labels, batched=True)
    return dataset


In [12]:
from datasets import load_dataset

def load_data():
    dataset = load_dataset('conll2003', trust_remote_code=True)
    return dataset


In [13]:
def preprocess_data(dataset):
    from transformers import XLMRobertaTokenizerFast

    tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")
    max_length = 128

    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["tokens"],
            truncation=True,
            padding="max_length",
            max_length=max_length,
            is_split_into_words=True,
        )
        
        labels = []
        for i, word_ids in enumerate(tokenized_inputs.word_ids(batch_index=batch_idx) for batch_idx in range(len(tokenized_inputs["input_ids"]))):
            example_labels = examples["ner_tags"][i]
            aligned_labels = []
            previous_word_id = None
            for word_id in word_ids:
                if word_id is None:
                    aligned_labels.append(-100)
                elif word_id != previous_word_id:
                    aligned_labels.append(example_labels[word_id])
                else:
                    aligned_labels.append(-100)
                previous_word_id = word_id
            
            labels.append(aligned_labels)

        tokenized_inputs["labels"] = labels
        return tokenized_inputs

    dataset = dataset.map(tokenize_and_align_labels, batched=True)
    return dataset


In [14]:
def main():
    # Load and preprocess the dataset
    dataset = load_data()
    dataset = preprocess_data(dataset)
    
    # Validation to ensure no mismatched input_ids and labels
    for split in ['train', 'validation', 'test']:
        for entry in dataset[split]:
            assert len(entry["input_ids"]) == len(entry["labels"]), f"Mismatch in {split} set"
    print("Dataset is valid and consistent.")

if __name__ == "__main__":
    main()


Map: 100%|██████████| 3453/3453 [00:00<00:00, 4026.36 examples/s]


Dataset is valid and consistent.


In [ ]:
Dataset Preprocessing and Validation Report
Objective:
To preprocess the dataset and validate its consistency for Named Entity Recognition (NER) using the XLM-RoBERTa model.

Steps Undertaken:

Data Preprocessing:
The dataset (ConLL 2003) was tokenized using XLM-RoBERTaTokenizerFast. Labels were aligned with tokenized inputs to ensure that they corresponded correctly to their respective tokens.

Padding and truncation were applied to standardize input lengths.
Subword tokenization was accounted for by ensuring special tokens and subwords were assigned a label of -100 or ignored during training.
Validation:
A thorough validation check was performed to ensure consistency:

Verified that the lengths of input_ids and labels matched for all examples in the train, validation, and test splits.
This confirmed that no misalignment or corruption occurred during the preprocessing stage.
Results:

The preprocessing step completed successfully across all splits:
Training Set: Processed 14,041 examples in ~4 seconds.
Validation Set: Processed 3,250 examples in ~1 second.
Test Set: Processed 3,453 examples in ~1 second.
Validation confirmed no mismatches or errors:
All examples had consistent lengths between input_ids and labels.
Conclusion:
The dataset is fully preprocessed and validated, ensuring it is ready for training the XLM-RoBERTa model. The successful preprocessing step provides confidence in the quality and integrity of the data for downstream tasks.



In [17]:
def preprocess_data(dataset):
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            padding="max_length",  # Pad to max length
            truncation=True,       # Truncate sequences exceeding max length
            max_length=128,        # Set max length for uniformity
            return_tensors="pt"    # Return PyTorch tensors
        )
    
    # Apply the tokenizer
    dataset = dataset.map(tokenize_function, batched=True)

    # Align labels with the tokenized data
    def align_labels(examples):
        max_len = 128  # Should match `max_length` used above
        aligned_labels = []
        for label in examples["labels"]:
            if len(label) > max_len:
                aligned_labels.append(label[:max_len])  # Truncate labels
            else:
                aligned_labels.append(label + [0] * (max_len - len(label)))  # Pad labels
        examples["labels"] = aligned_labels
        return examples
    
    dataset = dataset.map(align_labels, batched=True)

    # Set format for PyTorch
    dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
    return dataset


In [18]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",  # Replace with the model name you're using
    num_labels=2  # Adjust based on the number of labels in your dataset
)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
import pandas as pd

# Load the dataset correctly, assuming it's space-delimited (you can adjust the delimiter as needed)
file_path = r'C:\Users\user\Desktop\10 Acedamy W5\All Data\labeled_telegram_product_price_location.txt'

# If space-separated
df = pd.read_csv(file_path, delimiter='\s+', header=None, names=['word', 'label'])

# If tab-separated, use delimiter='\t'
# df = pd.read_csv(file_path, delimiter='\t', header=None, names=['word', 'label'])

# Display the first few rows to ensure it's loaded correctly
print(df.head())


            word label
0  Example_Group     O
1             is     O
2          being     O
3      protected     O
4             by     O


In [27]:
# Remove any rows with missing values (NaN)
df = df.dropna()

# Check the first few rows after cleaning
print(df.head())


            word label
0  Example_Group     O
1             is     O
2          being     O
3      protected     O
4             by     O


In [30]:
# Define a label map (example with 'O' and 'B-PRODUCT', you can expand this as needed)
label_map = {
    'O': 0,           # Outside entity
    'B-PRODUCT': 1,   # Beginning of a product entity
    'I-PRODUCT': 2,   # Inside of a product entity
    # Add other labels as needed...
}


In [31]:
# Convert labels from string to integer
df['label'] = df['label'].map(label_map)

# Check if the labels are properly converted
print(df.head())


            word  label
0  Example_Group      0
1             is      0
2          being      0
3      protected      0
4             by      0


In [4]:
from transformers import BertTokenizer
from datasets import Dataset
import pandas as pd

# Example dataset
data = {"word": ["Hello", "world", "this", "is", "a", "test"]}
df = pd.DataFrame(data)

# Initialize the tokenizer (you can use any available tokenizer)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenization and label alignment function
def tokenize_and_align_labels(example):
    # Tokenize the input words
    tokenized_inputs = tokenizer(example['word'], padding='max_length', truncation=True, max_length=512, is_split_into_words=True)
    
    # Ensure the tokenized length matches the expected length
    # Tokenize without padding and truncation first to understand original length
    word_ids = tokenized_inputs.word_ids(batch_index=0)  # Retrieves word-token mapping
    
    # Align the labels with tokenized inputs
    label_ids = []
    for word_id in word_ids:
        if word_id is None:
            label_ids.append(-100)  # Padding token is ignored
        else:
            label_ids.append(word_id)  # Align the word label

    tokenized_inputs["labels"] = label_ids
    return tokenized_inputs

# Convert the dataframe to a dataset
dataset = Dataset.from_pandas(df)

# Apply the tokenization and label alignment function
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Check the tokenized result for the first example
print(tokenized_dataset[0])


Map:   0%|          | 0/6 [00:00<?, ? examples/s]


ValueError: word_ids() is not available when using non-fast tokenizers (e.g. instance of a `XxxTokenizerFast` class).